In [40]:
import yfinance as yf
import pandas as pd

symbols = ["TSLA", "BND", "SPY"]
start = "2015-07-01"
end = "2025-07-31"

data = {}
for sym in symbols:
    df = yf.download(sym, start=start, end=end, progress=False, auto_adjust=False) 
    data[sym] = df
    print(sym, df.shape)
    display(df.head())


TSLA (2535, 6)


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,,
2015-07-01,17.943333,17.943333,18.174667,17.856667,18.073999,31518000
2015-07-02,18.667999,18.667999,18.830000,18.220667,18.680000,107458500
2015-07-06,18.648001,18.648001,18.779333,18.420000,18.591999,61828500
2015-07-07,17.858667,17.858667,18.346666,17.384666,18.333332,91576500
2015-07-08,16.997334,16.997334,17.386667,16.954000,17.288000,93316500


BND (2535, 6)


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BND,BND,BND,BND,BND,BND
Date,,,,,,
2015-07-01,60.816753,80.900002,81.029999,80.830002,80.870003,5399300
2015-07-02,60.967091,81.099998,81.180000,81.059998,81.059998,1060100
2015-07-06,61.177582,81.379997,81.440002,81.220001,81.440002,2210700
2015-07-07,61.237743,81.459999,81.769997,81.459999,81.690002,4183200
2015-07-08,61.410610,81.690002,81.760002,81.559998,81.639999,1967800


SPY (2535, 6)


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2015-07-01,174.917130,207.500000,208.029999,206.559998,207.729996,135979900
2015-07-02,174.756882,207.309998,208.270004,206.809998,208.070007,104373700
2015-07-06,174.259598,206.720001,207.649994,205.529999,205.770004,117975400
2015-07-07,175.355453,208.020004,208.169998,204.110001,206.960007,173820200
2015-07-08,172.413452,204.529999,206.759995,204.250000,206.419998,164020100


In [41]:
# df = df.reset_index()            # bring Date into columns
# df.columns = df.columns.droplevel(1) if isinstance(df.columns, pd.MultiIndex) else df.columns


# df.to_csv(f"../data/{sym}_ohlcv.csv", index=False)
# display(df.head())

df = df.reset_index()

# two header rows flatten it
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)  # keep only first level

# If there's an 'index' column from previous saves, drop it
if 'index' in df.columns:
    df = df.drop(columns=['index'])

# Ensure 'Date' exists and is the first column
cols = ['Date'] + [c for c in df.columns if c != 'Date']
df = df[cols]

# Save clean CSV
df.to_csv(f"../data/{sym}_ohlcv.csv", index=False)
display(df.head())


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2015-07-01,174.917130,207.500000,208.029999,206.559998,207.729996,135979900
1,2015-07-02,174.756882,207.309998,208.270004,206.809998,208.070007,104373700
2,2015-07-06,174.259598,206.720001,207.649994,205.529999,205.770004,117975400
3,2015-07-07,175.355453,208.020004,208.169998,204.110001,206.960007,173820200
4,2015-07-08,172.413452,204.529999,206.759995,204.250000,206.419998,164020100


In [ ]:
import pandas as pd
import os

data = {
    sym: pd.read_csv(
        os.path.join("../data", f"{sym}_ohlcv.csv"),
        parse_dates=["Date"],
        index_col="Date"
    )
    for sym in symbols
}



In [ ]:
for sym, df in data.items():
    print(f"\n=== {sym} ===")
    print(df.info())          # Column names, data types, non-null counts
    print(df.describe())      # Mean, std, min, quartiles, max


In [ ]:
for sym, df in data.items():
    print(f"{sym} missing values:\n", df.isnull().sum())


In [ ]:
for sym in data:
    data[sym] = data[sym].fillna(method="ffill")

In [ ]:
for sym in data:
    data[sym] = data[sym].interpolate()

In [ ]:
for sym in data:
    data[sym] = data[sym].dropna()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scalers = {}
for sym, df in data.items():
    scaler = MinMaxScaler()
    df[df.columns] = scaler.fit_transform(df)
    scalers[sym] = scaler  # store for inverse_transform later
    data[sym] = df


In [ ]:
clean_dir = "../data/cleaned"
os.makedirs(clean_dir, exist_ok=True)

for sym, df in data.items():
    df.to_csv(os.path.join(clean_dir, f"{sym}_ohlcv_clean.csv"))


In [ ]:
#EDA - Visualize Closing Price Over Time
import matplotlib.pyplot as plt

tsla = data['TSLA']

plt.figure(figsize=(12,6))
plt.plot(tsla.index, tsla['Close'], label='TSLA Close Price')
plt.title('TSLA Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()


In [ ]:
#Calculate & Plot Daily Percentage Change (Returns)
tsla['Daily_Return'] = tsla['Close'].pct_change() * 100  # percentage change

plt.figure(figsize=(12,6))
plt.plot(tsla.index, tsla['Daily_Return'], label='TSLA Daily Return (%)', color='orange')
plt.title('TSLA Daily Percentage Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return (%)')
plt.legend()
plt.show()


In [ ]:
#Analyze Volatility via Rolling Statistics
rolling_window = 30
tsla['Rolling_Mean'] = tsla['Close'].rolling(window=rolling_window).mean()
tsla['Rolling_Std'] = tsla['Close'].rolling(window=rolling_window).std()

plt.figure(figsize=(12,6))
plt.plot(tsla.index, tsla['Close'], label='TSLA Close Price')
plt.plot(tsla.index, tsla['Rolling_Mean'], label=f'{rolling_window}-Day Rolling Mean', linestyle='--')
plt.fill_between(tsla.index, 
                 tsla['Rolling_Mean'] - tsla['Rolling_Std'], 
                 tsla['Rolling_Mean'] + tsla['Rolling_Std'], 
                 color='grey', alpha=0.2, label='Rolling Std Dev')
plt.title('TSLA Price with Rolling Mean and Volatility')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()


In [ ]:
#Outlier Detection, major
mean_return = tsla['Daily_Return'].mean()
std_return = tsla['Daily_Return'].std()

outliers = tsla[(tsla['Daily_Return'] > mean_return + 3*std_return) | 
                (tsla['Daily_Return'] < mean_return - 3*std_return)]

print(f"Outlier days (extreme daily returns):\n{outliers[['Close', 'Daily_Return']]}")


In [ ]:
from statsmodels.tsa.stattools import adfuller

# ADF test on Close price
result_close = adfuller(tsla['Close'].dropna())
print('ADF Statistic (Close):', result_close[0])
print('p-value (Close):', result_close[1])

# ADF test on daily returns (usually stationary)
result_return = adfuller(tsla['Daily_Return'].dropna())
print('ADF Statistic (Daily Return):', result_return[0])
print('p-value (Daily Return):', result_return[1])

# Interpretation
if result_close[1] < 0.05:
    print("Close price series is stationary")
else:
    print("Close price series is non-stationary (needs differencing)")

if result_return[1] < 0.05:
    print("Daily returns are stationary (good for modeling)")
else:
    print("Daily returns are non-stationary")


In [ ]:
#Risk metrix
import numpy as np

# Value at Risk at 95% confidence
VaR_95 = np.percentile(tsla['Daily_Return'].dropna(), 5)
print(f"Value at Risk (95% confidence): {VaR_95:.2f}%")

# Sharpe Ratio
# Assuming risk-free rate ~0 (can adjust for realistic value)
mean_return_daily = tsla['Daily_Return'].mean()
std_return_daily = tsla['Daily_Return'].std()

sharpe_ratio = mean_return_daily / std_return_daily * np.sqrt(252)  # annualized
print(f"Annualized Sharpe Ratio: {sharpe_ratio:.2f}")
